In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings

#sklearn model
import optuna
from optuna.samplers import TPESampler
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

warnings.filterwarnings('ignore')

## Read Data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
train.head()

In [ ]:
FEATURES = [col for col in train.columns if col not in ['id', 'target']]

In [ ]:
print(f'train shape: {train.shape}')
print(f'test shape: {test.shape}')

In [ ]:
train.drop(['id', 'target'], axis=1).info()

In [ ]:
train.describe()

In [ ]:
x_train = train.drop(['id', 'target'], axis=1)
y_train = train.target

x_test = test.drop('id', axis=1)

In [ ]:
feature_std = x_train.std(axis=0)

In [ ]:
feature_std = feature_std / (x_train.max(axis=0) - x_train.min(axis=0))

In [ ]:
sns.histplot(feature_std)

In [ ]:
index = feature_std[feature_std< 0.1].index

In [ ]:
x_train.drop(index, axis=1).shape

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(2)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)

In [ ]:
plt.scatter(x_train_pca[:, 0], x_train_pca[:,1], c = y_train)

In [ ]:
del train, test
gc.collect()

## EDA

### data visualization

In [ ]:
# target visualization
plt.pie(y_train.value_counts(), labels=['One', 'Zero'], autopct='%1.1f%%')
plt.axis('equal') 

In [ ]:
# #feature visualization
# ncols = 3
# nrows = x_train.shape[1] // ncols + (x_train.shape[1] % ncols != 0)
# fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 5*nrows))

# for row in range(nrows):
#     for col in range(ncols):
#         index = row * ncols + col
#         if index >= x_train.shape[1] :
#             break
#         sns.kdeplot(x_train.iloc[:, index], ax=axes[row, col])
#         sns.kdeplot(x_test.iloc[:, index], ax=axes[row, col])

In [ ]:
# #feature visualization
# ncols = 3
# nrows = x_train.shape[1] // ncols + (x_train.shape[1] % ncols != 0)
# fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 5*nrows))

# for row in range(nrows):
#     for col in range(ncols):
#         index = row * ncols + col
#         if index >= x_train.shape[1] :
#             break
#         sns.kdeplot(x_train[y_train==1].iloc[:, index], ax=axes[row, col], label='One')
#         sns.kdeplot(x_train[y_train==0].iloc[:, index], ax=axes[row, col], label='Zero')
#         plt.legend()


In [ ]:
# np.sum((x_train[:, 96] > 2.5) == y_train) / y_train.shape[0]

In [ ]:
value = x_train.mean(axis=1)

In [ ]:
value = x_train.sum(axis=1)

In [ ]:
sns.kdeplot(value[y_train == 1])
sns.kdeplot(value[y_train == 0])

### feature engineering

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

y_train = y_train.values
gc.collect()

In [ ]:
feature_std = x_train.std(axis=0)

## Train Model

In [ ]:
# def objective(trial):

#     param_grid = {'objective': 'binary:logistic',
#               'use_label_encoder': False,
#               'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
#               'learning_rate': trial.suggest_discrete_uniform('learning_rate',0.01,0.1,0.01),
#               'subsample': trial.suggest_categorical ('subsample', [0.2,0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
#               'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree',0.1,1.0, 0.1),
#               'max_depth': trial.suggest_int('max_depth', 2, 20),
#               'booster': 'gbtree',
#               'gamma': trial.suggest_uniform('gamma',1.0,10.0),
#               'reg_alpha': trial.suggest_int('reg_alpha',50,100),
#               'reg_lambda': trial.suggest_int('reg_lambda',50,100),
#               'random_state': 42,
#                  }

#     x_train_, x_val, y_train_, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=50)
#     xgb_model = XGBClassifier(**param_grid, tree_method='gpu_hist', predictor='gpu_predictor',
#                             eval_metric=['logloss'])

#     xgb_model.fit(x_train_, y_train_, verbose=False)
#     y_pred = xgb_model.predict_proba(x_val)[:, 1]
#     return roc_auc_score(y_val, y_pred)

In [ ]:
# train_time = 1 * 30 * 60 # h * m * s
# study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBClassifier')
# study.optimize(objective, timeout=train_time)

# print('Number of finished trials: ', len(study.trials))
# print('Best trial:')
# trial = study.best_trial

# print('\tValue: {}'.format(trial.value))
# print('\tParams: ')
# for key, value in trial.params.items():
#     print('\t\t{}: {}'.format(key, value))

In [ ]:
# # xgb_params = trial.params
# xgb_params = {
#                 'n_estimators': 4492,
#                 'learning_rate': 0.01,
#                 'subsample': 1.0,
#                 'colsample_bytree': 0.2,
#                 'max_depth': 15,
#                 'gamma': 1.0328829988080024,
#                 'reg_alpha': 100,
#                 'reg_lambda': 93 }

# xgb_params['tree_method'] = 'gpu_hist'
# xgb_params['predictor'] = 'gpu_predictor'
# xgb_params['use_label_encoder'] = False

In [ ]:
# from sklearn.model_selection import KFold

# n_split = 10
# kfold = KFold(n_split)

# val_pred = np.zeros(y_train.shape)
# y_test = np.zeros((x_test.shape[0],))

# for i, (train_index, val_index) in enumerate(kfold.split(x_train)):
#     # train model
#     print("fold {} training".format(i))
#     model = XGBClassifier(**xgb_params, eval_metric=['logloss'])
#     model.fit(x_train[train_index], y_train[train_index])
    
#     # predict val and test
#     val_pred[val_index] = model.predict_proba(x_train[val_index])[:, 1]
#     vla_score = roc_auc_score(y_train[val_index], val_pred[val_index])
#     print("fold {} validation auc score {}".format(i, vla_score))
    
#     y_test += model.predict_proba(x_test)[:, 1] / n_split
    


### logic regretion 

In [ ]:
# from sklearn.linear_model import LogisticRegression

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)
# model = LogisticRegression()
# model.fit(x_train, y_train)
# y_pred = model.predict_proba(x_val)[:, 1]
# roc_auc_score(y_val, y_pred)

### svm

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
# def objective(trial):

#     param_grid = {
#               'tol': trial.suggest_categorical('tol', [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]),
#               'max_iter': trial.suggest_int('max_iter',1000, 5000)
#                  }

#     x_train_, x_val, y_train_, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=50)
#     svc = LinearSVC(**param_grid, penalty='l2', dual=False)
    
#     svc.fit(x_train_, y_train_)
#     val_pred_svc = svc.decision_function(x_val)
#     return roc_auc_score(y_val, val_pred_svc)

In [ ]:
# train_time = 1 * 30 * 60 # h * m * s
# study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='svc')
# study.optimize(objective, timeout=train_time)

# print('Number of finished trials: ', len(study.trials))
# print('Best trial:')
# trial = study.best_trial

# print('\tValue: {}'.format(trial.value))
# print('\tParams: ')
# for key, value in trial.params.items():
#     print('\t\t{}: {}'.format(key, value))

In [ ]:
# svc_params = trial.params

In [ ]:
# from sklearn.model_selection import KFold

# n_split = 10
# kfold = KFold(n_split)

# val_pred = np.zeros(y_train.shape)
# y_test = np.zeros((x_test.shape[0],))

# for i, (train_index, val_index) in enumerate(kfold.split(x_train)):
#     # train model
#     print("fold {} training".format(i))
    
#     svc = LinearSVC(**svc_params, penalty='l2', dual=False)
#     svc.fit(x_train[train_index], y_train[train_index])
    
#     # predict val and test
#     val_pred[val_index] = svc.decision_function(x_train[val_index])
#     vla_score = roc_auc_score(y_train[val_index], val_pred[val_index])
#     print("fold {} validation auc score {}".format(i, vla_score))
    
#     y_test += svc.decision_function(x_test) / n_split
    

In [ ]:
x_data = x_train
y_data = y_train

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

svc = LinearSVC(tol=1e-6, penalty='l2', dual=False, max_iter=1900)
svc.fit(x_train, y_train)
y_pred = svc.decision_function(x_val)
score = roc_auc_score(y_val, y_pred)

# print('val auc score :', score)
# y_test = svc.decision_function(x_test)[:, 1]
# print(y_test[:10])

In [ ]:
print(score)

In [ ]:
y_pred_binary = svc.predict(x_data)

In [ ]:
np.sum(y_pred_binary == y_data) / y_data.shape[0]

In [ ]:
y_pred_data = svc.decision_function(x_data)

In [ ]:
sns.kdeplot(y_pred_data[y_data == 1], label='One')
sns.kdeplot(y_pred_data[y_data == 0], label='Zero')
plt.legend()

In [ ]:
np.sum((y_pred_data > 0).astype('int') == y_data) / y_data.shape[0]

In [ ]:
np.sum(y_pred_data > 0) / y_pred_data.shape[0]

In [ ]:
1 - (np.sum(y_pred_data[y_data == 1] < 0) + np.sum(y_pred_data[y_data == 0] > 0)) / y_pred_data.shape[0]

In [ ]:
(np.sum(y_pred_data[y_data == 0] < 0)) / np.sum(y_pred_data < 0)

In [ ]:
print(score)

In [ ]:
y_data_pred = svc.predict(x_data)
y_data_proba = svc.decision_function(x_data)

y_data_proba_right = y_data_proba[y_data == y_data_pred]
y_data_proba_wrong = y_data_proba[y_data != y_data_pred]

import seaborn as sns

sns.histplot(y_data_proba_right, label='right')
plt.legend()
plt.figure()
sns.histplot(y_data_proba_wrong, label='wrong')
plt.legend()

In [ ]:
plt.scatter(y_pred_data, np.zeros(y_pred_data.shape), c=y_data)
plt.legend()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

x_train, x_val, y_train, y_val = train_test_split(y_pred_data.reshape((-1, 1)), y_data, test_size=0.1)
decision_tree = DecisionTreeClassifier(max_depth=20)

decision_tree.fit(y_pred_data.reshape((-1, 1)), y_data)
y_pred_train = decision_tree.predict_proba(x_train)[:, 1]
train_score = roc_auc_score(y_train, y_pred_train)
print(f'roc_auc_score train score {train_score}')

y_pred = decision_tree.predict_proba(x_val)[:, 1]
val_score = roc_auc_score(y_val, y_pred)
print(f'roc_auc_score val score {val_score}')


In [ ]:
y_pred = svc.decision_function(x_test)

In [ ]:
y_test = decision_tree.predict_proba(y_pred.reshape((-1, 1)))[:, 1]

In [ ]:
y_test.shape

In [ ]:
# x_data_1 = x_data[np.abs(y_data_proba) < 0.1]
# y_data_1 = y_data[np.abs(y_data_proba) < 0.1]

In [ ]:
# y_val_proba = svc.decision_function(x_val)

# y_pred = svc.decision_function(x_val[np.abs(y_val_proba) > 0.05])
# score = roc_auc_score(y_val[np.abs(y_val_proba) > 0.05], y_pred)
# print(score)

In [ ]:
# x_train, x_val, y_train, y_val = train_test_split(x_data_1, y_data_1)

# svc1 = LinearSVC(tol=1e-6, penalty='l2', dual=False, max_iter=1900)
# svc1.fit(x_train, y_train)
# y_pred = svc1.decision_function(x_val)
# score = roc_auc_score(y_val, y_pred)
# print(score)

### tree

In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

# svc1 = DecisionTreeClassifier()
# svc1.fit(x_train, y_train)
# y_pred = svc1.predict_proba(x_val)[:,1]
# score = roc_auc_score(y_val, y_pred)

# print('val auc score :', score)
# y_test = svc.predict_proba(x_test)[:, 1]
# print(y_test[:10])

## Validation Score

In [ ]:
# print("val auc score :", roc_auc_score(y_train, val_pred))

## Submission

In [ ]:
sub_mission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
sub_mission.target = y_test
sub_mission.to_csv('submission.csv', index=False)


In [ ]:
plt.figure()
sns.kdeplot(y_test)